In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [7]:
data = requests.get(standings_url)

In [3]:
from bs4 import BeautifulSoup

In [8]:
soup = BeautifulSoup(data.text)

In [9]:
standings_table = soup.select('table.stats_table')[0]

In [11]:
links = standings_table.find_all('a')

In [12]:
links = [l.get("href") for l in links]

In [13]:
links = [l for l in links if '/squads/' in l]

In [15]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [18]:
team_url = team_urls[0]

In [19]:
data = requests.get(team_url)

In [4]:
import pandas as pd

In [23]:
matches = pd.read_html(data.text, match="Scores & Fixtures")

/var/folders/14/7g5dmrxj7lxbz8__3gfs4lfr0000gn/T/ipykernel_98819/1970929637.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")


In [26]:
soup = BeautifulSoup(data.text)

In [27]:
links = soup.find_all('a')

In [28]:
links = [l.get("href") for l in links]

In [29]:
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [30]:
links

['/en/squads/b8fd03ef/2024-2025/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2024-2025/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2024-2025/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2024-2025/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions']

In [31]:
data = requests.get(f"https://fbref.com{links[0]}")

In [33]:
shooting = pd.read_html(data.text, match="Shooting")[0]

/var/folders/14/7g5dmrxj7lxbz8__3gfs4lfr0000gn/T/ipykernel_98819/2517714723.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [35]:
shooting.columns = shooting.columns.droplevel()

In [36]:
shooting

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2024-08-10,15:00,FA Community Shield,FA Community Shield,Sat,Home,D,1 (7),1 (6),Manchester Utd,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,Chelsea,...,19.1,0.0,0,0,0.8,0.8,0.07,1.2,1.2,Match Report
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4,1,Ipswich Town,...,17.8,1.0,1,1,3.3,2.6,0.20,0.7,0.4,Match Report
3,2024-08-31,17:30,Premier League,Matchweek 3,Sat,Away,W,3,1,West Ham,...,15.0,1.0,0,0,3.0,3.0,0.13,0.0,0.0,Match Report
4,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,W,2,1,Brentford,...,17.2,0.0,0,0,2.1,2.1,0.12,-0.1,-0.1,Match Report
5,NaN,NaN,NaN,NaN,NaN,NaN,--,NaN,NaN,NaN,...,16.9,2.0,1,1,NaN,NaN,0.13,12.0,11.0,NaN


In [37]:
team_data = matches[0].merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]], on="Date")

In [38]:
team_data


,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2024-08-10,15:00,FA Community Shield,FA Community Shield,Sat,Home,D,1 (7),1 (6),Manchester Utd,...,4-2-3-1,Jarred Gillett,Match Report,Manchester City won on penalty kicks following...,9,1,NaN,NaN,0,0
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,Chelsea,...,4-2-3-1,Anthony Taylor,Match Report,NaN,11,5,19.1,0.0,0,0
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4,1,Ipswich Town,...,5-4-1,Samuel Allison,Match Report,NaN,13,4,17.8,1.0,1,1
3,2024-08-31,17:30,Premier League,Matchweek 3,Sat,Away,W,3,1,West Ham,...,4-2-3-1,Michael Oliver,Match Report,NaN,23,8,15.0,1.0,0,0
4,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,W,2,1,Brentford,...,5-3-2,Darren Bond,Match Report,NaN,18,7,17.2,0.0,0,0


In [5]:
years = list(range(2025,2022, -1))

In [6]:
all_matches = []

In [7]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [17]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        try:
            shooting = pd.read_html(data.text, match="Shooting")[0]
            shooting.columns = shooting.columns.droplevel()
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        time.sleep(10)
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)

    time.sleep(10)
        

/var/folders/14/7g5dmrxj7lxbz8__3gfs4lfr0000gn/T/ipykernel_99777/833923119.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/14/7g5dmrxj7lxbz8__3gfs4lfr0000gn/T/ipykernel_99777/833923119.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
/var/folders/14/7g5dmrxj7lxbz8__3gfs4lfr0000gn/T/ipykernel_99777/833923119.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/14/7g5dmrxj7lxbz8__3gfs4lfr0000gn/T/ipykerne

In [18]:
match_df = pd.concat(all_matches)

In [19]:
match_df.columns = [c.lower() for c in match_df.columns]

In [20]:
match_df.to_csv("matches.csv")